# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f260b972af0>
├── 'a' --> tensor([[ 0.1714, -0.7040, -0.0253],
│                   [-0.2833,  1.4810,  0.4034]])
└── 'x' --> <FastTreeValue 0x7f260b972a30>
    └── 'c' --> tensor([[-0.0704, -0.1894, -0.9141,  1.5382],
                        [-0.6836, -2.3312,  0.8132,  0.1927],
                        [ 1.2177, -2.0872, -1.3261, -1.0642]])

In [4]:
t.a

tensor([[ 0.1714, -0.7040, -0.0253],
        [-0.2833,  1.4810,  0.4034]])

In [5]:
%timeit t.a

76.5 ns ± 1.12 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f260b972af0>
├── 'a' --> tensor([[ 0.8381, -0.1267,  0.7325],
│                   [ 1.6251, -0.0650, -2.0804]])
└── 'x' --> <FastTreeValue 0x7f260b972a30>
    └── 'c' --> tensor([[-0.0704, -0.1894, -0.9141,  1.5382],
                        [-0.6836, -2.3312,  0.8132,  0.1927],
                        [ 1.2177, -2.0872, -1.3261, -1.0642]])

In [7]:
%timeit t.a = new_value

80.7 ns ± 1.11 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1714, -0.7040, -0.0253],
               [-0.2833,  1.4810,  0.4034]]),
    x: Batch(
           c: tensor([[-0.0704, -0.1894, -0.9141,  1.5382],
                      [-0.6836, -2.3312,  0.8132,  0.1927],
                      [ 1.2177, -2.0872, -1.3261, -1.0642]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1714, -0.7040, -0.0253],
        [-0.2833,  1.4810,  0.4034]])

In [11]:
%timeit b.a

68.3 ns ± 1.43 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3120, -0.7845,  1.4945],
               [ 0.5157,  1.2948, -0.2645]]),
    x: Batch(
           c: tensor([[-0.0704, -0.1894, -0.9141,  1.5382],
                      [-0.6836, -2.3312,  0.8132,  0.1927],
                      [ 1.2177, -2.0872, -1.3261, -1.0642]]),
       ),
)

In [13]:
%timeit b.a = new_value

658 ns ± 3.63 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.1 µs ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.6 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

295 µs ± 3.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

286 µs ± 7.41 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f255c5857c0>
├── 'a' --> tensor([[[ 0.1714, -0.7040, -0.0253],
│                    [-0.2833,  1.4810,  0.4034]],
│           
│                   [[ 0.1714, -0.7040, -0.0253],
│                    [-0.2833,  1.4810,  0.4034]],
│           
│                   [[ 0.1714, -0.7040, -0.0253],
│                    [-0.2833,  1.4810,  0.4034]],
│           
│                   [[ 0.1714, -0.7040, -0.0253],
│                    [-0.2833,  1.4810,  0.4034]],
│           
│                   [[ 0.1714, -0.7040, -0.0253],
│                    [-0.2833,  1.4810,  0.4034]],
│           
│                   [[ 0.1714, -0.7040, -0.0253],
│                    [-0.2833,  1.4810,  0.4034]],
│           
│                   [[ 0.1714, -0.7040, -0.0253],
│                    [-0.2833,  1.4810,  0.4034]],
│           
│                   [[ 0.1714, -0.7040, -0.0253],
│                    [-0.2833,  1.4810,  0.4034]]])
└── 'x' --> <FastTreeValue 0x7f255cb7fbb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

50.2 µs ± 983 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f255c585730>
├── 'a' --> tensor([[ 0.1714, -0.7040, -0.0253],
│                   [-0.2833,  1.4810,  0.4034],
│                   [ 0.1714, -0.7040, -0.0253],
│                   [-0.2833,  1.4810,  0.4034],
│                   [ 0.1714, -0.7040, -0.0253],
│                   [-0.2833,  1.4810,  0.4034],
│                   [ 0.1714, -0.7040, -0.0253],
│                   [-0.2833,  1.4810,  0.4034],
│                   [ 0.1714, -0.7040, -0.0253],
│                   [-0.2833,  1.4810,  0.4034],
│                   [ 0.1714, -0.7040, -0.0253],
│                   [-0.2833,  1.4810,  0.4034],
│                   [ 0.1714, -0.7040, -0.0253],
│                   [-0.2833,  1.4810,  0.4034],
│                   [ 0.1714, -0.7040, -0.0253],
│                   [-0.2833,  1.4810,  0.4034]])
└── 'x' --> <FastTreeValue 0x7f255c585cd0>
    └── 'c' --> tensor([[-0.0704, -0.1894, -0.9141,  1.5382],
                        [-0.6836, -2.3312,  0.8132,  0.1927],
                 

In [23]:
%timeit t_cat(trees)

46.4 µs ± 440 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

85.8 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.1714, -0.7040, -0.0253],
                [-0.2833,  1.4810,  0.4034]],
       
               [[ 0.1714, -0.7040, -0.0253],
                [-0.2833,  1.4810,  0.4034]],
       
               [[ 0.1714, -0.7040, -0.0253],
                [-0.2833,  1.4810,  0.4034]],
       
               [[ 0.1714, -0.7040, -0.0253],
                [-0.2833,  1.4810,  0.4034]],
       
               [[ 0.1714, -0.7040, -0.0253],
                [-0.2833,  1.4810,  0.4034]],
       
               [[ 0.1714, -0.7040, -0.0253],
                [-0.2833,  1.4810,  0.4034]],
       
               [[ 0.1714, -0.7040, -0.0253],
                [-0.2833,  1.4810,  0.4034]],
       
               [[ 0.1714, -0.7040, -0.0253],
                [-0.2833,  1.4810,  0.4034]]]),
    x: Batch(
           c: tensor([[[-0.0704, -0.1894, -0.9141,  1.5382],
                       [-0.6836, -2.3312,  0.8132,  0.1927],
                       [ 1.2177, -2.0872, -1.3261, -1.0642]],
         

In [26]:
%timeit Batch.stack(batches)

110 µs ± 1.85 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.1714, -0.7040, -0.0253],
               [-0.2833,  1.4810,  0.4034],
               [ 0.1714, -0.7040, -0.0253],
               [-0.2833,  1.4810,  0.4034],
               [ 0.1714, -0.7040, -0.0253],
               [-0.2833,  1.4810,  0.4034],
               [ 0.1714, -0.7040, -0.0253],
               [-0.2833,  1.4810,  0.4034],
               [ 0.1714, -0.7040, -0.0253],
               [-0.2833,  1.4810,  0.4034],
               [ 0.1714, -0.7040, -0.0253],
               [-0.2833,  1.4810,  0.4034],
               [ 0.1714, -0.7040, -0.0253],
               [-0.2833,  1.4810,  0.4034],
               [ 0.1714, -0.7040, -0.0253],
               [-0.2833,  1.4810,  0.4034]]),
    x: Batch(
           c: tensor([[-0.0704, -0.1894, -0.9141,  1.5382],
                      [-0.6836, -2.3312,  0.8132,  0.1927],
                      [ 1.2177, -2.0872, -1.3261, -1.0642],
                      [-0.0704, -0.1894, -0.9141,  1.5382],
                      [-0.6836, -

In [28]:
%timeit Batch.cat(batches)

237 µs ± 3.57 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

558 µs ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
